In [2]:
import os
# 【硬性要求】加载环境变量，确保能读取到 OPENAI_API_KEY
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

# ==========================================
# 1. 初始化带有“可配置字段”的模型
# ==========================================
# 【科研级保姆注释】
# 旧版本：ChatOpenAI(model_name="gpt-3.5-turbo")
# 现代写法：推荐直接使用 `model` 参数。
# 我们通过 .configurable_fields() 为其绑定一个配置钩子。
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7).configurable_fields(
    # 将内部的 "model" 参数暴露为可配置的字段
    model_name=ConfigurableField(
        id="llm_model_id", # 这个 ID 是你在运行时调用的钥匙
        name="LLM Model Name",
        description="The GPT model version to use for this specific task",
    )
)

# ==========================================
# 2. 构建基础提示词与链
# ==========================================
prompt = PromptTemplate.from_template("Write a Haiku (俳句) on {subject}")
chain = prompt | model

# ==========================================
# 3. 动态调用演示
# ==========================================
if __name__ == "__main__":
    subject_topic = "quantum physics" # 用量子物理写首诗
    
    print("=== 1. 使用默认模型 (GPT-3.5-turbo) ===")
    # 直接 invoke，触发默认设置
    default_response = chain.invoke({"subject": subject_topic})
    print(f"[模型]: {default_response.response_metadata.get('model_name')}")
    print(f"[输出]:\n{default_response.content}\n")
    
    print("=== 2. 运行时动态切换模型 (GPT-4o) ===")
    # 【核心魔法】：使用 .with_config() 在运行时注入新参数
    # 这里的键 "llm_model_id" 必须与上面 ConfigurableField 定义的 id 完全一致
    advanced_response = chain.with_config(
        configurable={"llm_model_id": "gpt-4o"}
    ).invoke({"subject": subject_topic})
    
    print(f"[模型]: {advanced_response.response_metadata.get('model_name')}")
    print(f"[输出]:\n{advanced_response.content}")

=== 1. 使用默认模型 (GPT-3.5-turbo) ===
[模型]: gpt-3.5-turbo-0125
[输出]:
Particles dance in
waves of probability
Reality shifts

=== 2. 运行时动态切换模型 (GPT-4o) ===
[模型]: gpt-4o-2024-08-06
[输出]:
Particles whisper,  
Entangled in dance of light—  
Worlds in a blink merge.
